# SQLite Persistence with Pause & Resume: Complete Lab

## 📚 Module Overview

This standalone notebook demonstrates **file-based persistence** in LangGraph using SQLite, with full **pause and resume** capabilities.

**What You'll Learn:**
- Create file-based SQLite databases for persistence
- Save conversation state to disk
- Pause and close the application
- Resume conversations after restart
- Inspect database contents
- Build production-ready HR onboarding agents

**Real-World Scenario:**
An HR onboarding assistant that works across multiple days:
- **Day 1:** Employee starts onboarding, completes some steps
- **Night:** System shuts down, state saved to database
- **Day 2:** Employee returns, conversation continues seamlessly

**Time:** 45-60 minutes

---

## 🔧 Step 1: Install LangChain & LangGraph 1.0 Alpha Packages

**Important:** We're using LangChain 1.0 alpha versions for the latest features.

**Note:** This may take 2-3 minutes. Run this cell first!

In [ ]:
# Install LangChain 1.0 Alpha packages
# Using --pre flag to get pre-release versions

!pip install --pre -U langchain langchain-openai langgraph langchain-community --quiet
!pip install langgraph-checkpoint-sqlite --quiet

print("✅ All packages installed successfully!")

In [ ]:
# Verify installations
import langchain
import langgraph

print("📦 Package Versions:")
print(f"   - LangChain: {langchain.__version__}")
print(f"   - LangGraph: {langgraph.__version__}")
print("\n✅ All imports successful!")
print("\n💡 Using LangChain 1.0 Alpha API")

## 🔑 Step 2: Configure OpenAI API Key

**Choose your environment:**
- **Google Colab:** Uses Colab secrets
- **Jupyter/Local:** Uses environment variable or input

In [ ]:
import os
import sys

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Google Colab - use secrets
    try:
        from google.colab import userdata
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
        print("✅ API Key loaded from Colab secrets")
    except Exception as e:
        print("⚠️  Please add OPENAI_API_KEY to Colab secrets")
        print("   Go to: 🔑 icon on left sidebar > Add new secret")
        raise
else:
    # Local Jupyter - check environment or prompt
    if 'OPENAI_API_KEY' not in os.environ:
        print("🔑 OpenAI API Key not found in environment")
        import getpass
        api_key = getpass.getpass("Please enter your OpenAI API Key: ")
        os.environ['OPENAI_API_KEY'] = api_key
    print("✅ API Key configured")

# Verify API key is set
if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError("OPENAI_API_KEY not set!")

print("\n✅ OpenAI API Key configured successfully!")

## 📥 Step 3: Import Required Libraries

In [ ]:
# Core LangChain and LangGraph imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# LangGraph checkpointing
from langgraph.checkpoint.sqlite import SqliteSaver

# Standard library
import sqlite3
import os
from typing import Annotated

print("✅ All libraries imported successfully!")

## 🗄️ Step 4: Setup SQLite Database

We'll create a **file-based** SQLite database that persists to disk.

In [ ]:
# Database file configuration
DB_FILE = "hr_assistant_persistent.db"

# Remove old database for fresh start (optional)
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"🗑️  Removed existing database: {DB_FILE}")

print(f"\n📁 Database file: {os.path.abspath(DB_FILE)}")
print("✅ Ready to create persistent SQLite database!")

## 🛠️ Step 5: Define HR Onboarding Tools

These tools will be used by our HR assistant agent.

In [ ]:
# Enhanced HR tools for onboarding workflow

@tool
def verify_documents(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Verify that employee has submitted all required documents."""
    # Simulated document check
    documents = ["ID Proof", "Address Proof", "Educational Certificates", "Previous Employment Letter"]
    return f"✅ All documents verified for employee {employee_id}: {', '.join(documents)}"

@tool
def assign_equipment(employee_id: Annotated[str, "Employee ID"], 
                     department: Annotated[str, "Department name"]) -> str:
    """Assign laptop and equipment to employee based on department."""
    equipment = {
        "Engineering": "MacBook Pro 16\", Monitor, Keyboard, Mouse",
        "Marketing": "Dell Laptop, Monitor, Marketing Kit",
        "Sales": "Dell Laptop, Mobile Phone, Sales Materials",
        "HR": "Dell Laptop, Monitor, HR Software Access"
    }
    assigned = equipment.get(department, "Standard Laptop Package")
    return f"✅ Equipment assigned to {employee_id}: {assigned}"

@tool
def schedule_orientation(employee_id: Annotated[str, "Employee ID"],
                        employee_name: Annotated[str, "Employee name"]) -> str:
    """Schedule orientation session for new employee."""
    return f"✅ Orientation scheduled for {employee_name} ({employee_id}):\n" \
           f"   📅 Date: Monday, 9:00 AM\n" \
           f"   📍 Location: Conference Room A\n" \
           f"   ⏱️  Duration: 3 hours"

@tool
def setup_accounts(employee_id: Annotated[str, "Employee ID"],
                  email: Annotated[str, "Employee email"]) -> str:
    """Setup email and system accounts for employee."""
    return f"✅ Accounts created for {employee_id}:\n" \
           f"   📧 Email: {email}\n" \
           f"   🔐 VPN Access: Enabled\n" \
           f"   💻 System Login: Created\n" \
           f"   📱 Slack: Account created"

# Collect all tools
onboarding_tools = [
    verify_documents,
    assign_equipment,
    schedule_orientation,
    setup_accounts
]

print("✅ HR Onboarding tools defined:")
for tool in onboarding_tools:
    print(f"   - {tool.name}: {tool.description}")

---
# 🚀 SESSION 1: Initial Onboarding (Day 1)

**Scenario:** Employee Priya starts onboarding on Day 1

## Step 6: Create Agent with SQLite Persistence

In [ ]:
print("="*80)
print("🚀 SESSION 1: Starting Employee Onboarding (Day 1)")
print("="*80)

# Create SQLite connection to FILE (not :memory:)
connection = sqlite3.connect(DB_FILE, check_same_thread=False)
checkpointer = SqliteSaver(connection)

print(f"\n✅ Connected to database: {DB_FILE}")
print(f"📊 Database location: {os.path.abspath(DB_FILE)}")

# Create agent with persistent checkpointing
# Note: NO 'prompt' parameter in LangChain 1.0 Alpha
persistent_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=onboarding_tools,
    checkpointer=checkpointer
)

# Thread ID for this employee's onboarding
config = {"configurable": {"thread_id": "employee_priya_onboarding_2024"}}

print(f"\n✅ Agent created with SQLite checkpointer")
print(f"🆔 Thread ID: employee_priya_onboarding_2024")
print("\n" + "-"*80)

## Step 7: Turn 1 - Employee Introduction

In [ ]:
print("\n👤 TURN 1: Employee Introduction")
print("-"*80)

# First invoke - include system message to set agent behavior
result = persistent_agent.invoke(
    {"messages": [
        {
            "role": "system",
            "content": """You are a helpful HR Onboarding Assistant.
            
Your responsibilities:
- Guide new employees through the onboarding process
- Verify documents
- Assign equipment
- Schedule orientation
- Setup accounts

Be friendly, professional, and remember all conversation context.
Guide employees step-by-step through onboarding."""
        },
        {
            "role": "user",
            "content": "Hi! I'm Priya Sharma, employee ID EMP-2024-101. I'm joining the Engineering department today and need to complete my onboarding."
        }
    ]},
    config
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)
print(f"\n💾 State saved to: {DB_FILE}")

## Step 8: Turn 2 - Document Verification

In [ ]:
print("\n📄 TURN 2: Document Verification")
print("-"*80)

# Subsequent turns - no need for system message (it's remembered!)
result = persistent_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Yes, I've uploaded all my documents. Can you verify them?"
    }]},
    config
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)
print(f"\n💾 State updated in: {DB_FILE}")

## Step 9: Turn 3 - Equipment Assignment

In [ ]:
print("\n💻 TURN 3: Equipment Assignment")
print("-"*80)

result = persistent_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Great! Now I need my laptop and equipment for the Engineering department."
    }]},
    config
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)
print(f"\n💾 State saved to: {DB_FILE}")

## Step 10: Check Current State

In [ ]:
print("\n" + "="*80)
print("📊 CHECKING STATE BEFORE PAUSE")
print("="*80)

# Get current state
current_state = persistent_agent.get_state(config)

print(f"\n✅ Current state saved successfully!")
print(f"\n📝 State Information:")
print(f"   - Total messages in conversation: {len(current_state.values['messages'])}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")

# Show conversation summary
print(f"\n💬 Conversation Summary:")
for i, msg in enumerate(current_state.values['messages'], 1):
    role = msg.__class__.__name__
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"   {i}. {role}: {content}")

## Step 11: PAUSE - Simulate End of Day / Application Shutdown

**This simulates:**
- Employee going home
- Application shutting down
- Database connection closing
- All state saved to disk

In [ ]:
print("\n" + "="*80)
print("⏸️  PAUSING SESSION - Simulating End of Day / Application Shutdown")
print("="*80)

# Verify database file exists
print(f"\n✅ Database file exists: {os.path.exists(DB_FILE)}")
print(f"📁 File location: {os.path.abspath(DB_FILE)}")
print(f"💾 File size: {os.path.getsize(DB_FILE)} bytes")

# Close connection (simulate application shutdown)
connection.close()
print(f"\n🔌 Database connection closed")
print(f"\n⏸️  SESSION 1 PAUSED")
print(f"\n💡 The conversation state is now saved to disk in: {DB_FILE}")
print(f"💡 We can now restart the application and resume the conversation!")

# Clean up variables to simulate fresh start
del persistent_agent
del checkpointer
print(f"\n🧹 Cleaned up agent and checkpointer from memory")
print(f"\n" + "="*80)

## Step 12: (Optional) Inspect Database Contents

**Let's peek inside the SQLite database to see what's stored!**

In [ ]:
print("\n" + "="*80)
print("🔍 INSPECTING DATABASE CONTENTS")
print("="*80)

# Open database for inspection only (read-only)
inspect_conn = sqlite3.connect(DB_FILE)
cursor = inspect_conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"\n📋 Tables in database:")
for table in tables:
    print(f"   - {table[0]}")

# Count checkpoints
cursor.execute("SELECT COUNT(*) FROM checkpoints;")
checkpoint_count = cursor.fetchone()[0]
print(f"\n💾 Total checkpoints stored: {checkpoint_count}")

# Show checkpoint details
cursor.execute("""
    SELECT thread_id, checkpoint_id, parent_checkpoint_id
    FROM checkpoints
    ORDER BY checkpoint_id
    LIMIT 5;
""")
checkpoints = cursor.fetchall()

print(f"\n📝 Recent Checkpoints:")
for i, (thread_id, checkpoint_id, parent_id) in enumerate(checkpoints, 1):
    print(f"   {i}. Thread: {thread_id[:30]}...")
    print(f"      Checkpoint: {checkpoint_id[:24]}...")
    print(f"      Parent: {parent_id[:24] if parent_id else 'None'}...\n")

# Count writes
cursor.execute("SELECT COUNT(*) FROM writes;")
writes_count = cursor.fetchone()[0]
print(f"💾 Total writes stored: {writes_count}")

cursor.close()
inspect_conn.close()

print(f"\n✅ Database inspection complete")
print(f"\n" + "="*80)

---
# 🔄 SESSION 2: Resume After Restart (Day 2)

**Scenario:** Next day - Employee returns, application restarts, conversation resumes

## Step 13: Reconnect to Database and Create New Agent

In [ ]:
print("\n" + "="*80)
print("🔄 SESSION 2: Resuming Employee Onboarding (Day 2)")
print("="*80)
print("\n⏰ Simulating: Next day, application restarted, employee returns...")

# Reconnect to the SAME database file
new_connection = sqlite3.connect(DB_FILE, check_same_thread=False)
new_checkpointer = SqliteSaver(new_connection)

print(f"\n✅ Reconnected to database: {DB_FILE}")
print(f"✅ Loaded existing checkpointer")

# Create NEW agent instance (simulating fresh application start)
resumed_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=onboarding_tools,
    checkpointer=new_checkpointer  # Uses the SAME database!
)

print(f"\n✅ Created new agent instance (fresh start)")
print(f"\n🔗 Using SAME thread ID to resume conversation")
print(f"\n" + "-"*80)

## Step 14: Retrieve Previous State

In [ ]:
print("\n📥 RETRIEVING PREVIOUS STATE")
print("-"*80)

# Use the SAME config (thread_id) to retrieve state
config = {"configurable": {"thread_id": "employee_priya_onboarding_2024"}}

retrieved_state = resumed_agent.get_state(config)

print(f"\n✅ Successfully retrieved previous state!")
print(f"\n📊 Retrieved State Information:")
print(f"   - Messages in history: {len(retrieved_state.values['messages'])}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")

print(f"\n💬 Previous Conversation Recap (from Day 1):")
print(f"-"*80)
for i, msg in enumerate(retrieved_state.values['messages'], 1):
    role = msg.__class__.__name__
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"\n{i}. {role}:")
    print(f"   {content}")

print(f"\n" + "-"*80)
print(f"\n✅ The agent remembers EVERYTHING from yesterday!")

## Step 15: Turn 4 - Continue Onboarding (Orientation)

In [ ]:
print("\n📅 TURN 4: Continuing Onboarding - Orientation Scheduling")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Thanks! I received my equipment. Can you schedule my orientation session?"
    }]},
    config  # Same thread_id - continues from where we left off!
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)
print(f"\n✅ Agent continued seamlessly from previous session!")

## Step 16: Turn 5 - Account Setup

In [ ]:
print("\n🔐 TURN 5: Account Setup")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Perfect! Now I need my email and system accounts. My email should be priya.sharma@company.com"
    }]},
    config
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)

## Step 17: Turn 6 - Test Memory (Recall from Day 1)

In [ ]:
print("\n🧠 TURN 6: Testing Memory - Recall from Day 1")
print("-"*80)

result = resumed_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Can you remind me what we did yesterday and what my employee ID is?"
    }]},
    config
)

print(f"\n🤖 Assistant:")
print(result['messages'][-1].content)
print(f"\n✅ Agent remembered details from PREVIOUS SESSION (Day 1)!")

## Step 18: Final State Check and Cleanup

In [ ]:
print("\n" + "="*80)
print("📊 FINAL STATE CHECK")
print("="*80)

final_state = resumed_agent.get_state(config)

print(f"\n✅ Onboarding Session Complete!")
print(f"\n📝 Final Statistics:")
print(f"   - Total messages exchanged: {len(final_state.values['messages'])}")
print(f"   - Thread ID: {config['configurable']['thread_id']}")

# Get state history to show all checkpoints
state_history = list(resumed_agent.get_state_history(config))
print(f"\n💾 Total checkpoints created: {len(state_history)}")
print(f"\n📈 Checkpoint progression:")
for i, state in enumerate(state_history[:5], 1):
    msg_count = len(state.values.get('messages', []))
    print(f"   Checkpoint {i}: {msg_count} messages")

if len(state_history) > 5:
    print(f"   ... and {len(state_history) - 5} more checkpoints")

In [ ]:
# Close connection
new_connection.close()
print(f"\n🔌 Database connection closed")
print(f"\n💾 All data safely persisted in: {os.path.abspath(DB_FILE)}")
print(f"\n" + "="*80)

## Step 19: Verification - Prove Persistence Works!

In [ ]:
print("\n" + "="*80)
print("🔬 VERIFICATION: Demonstrating True Persistence")
print("="*80)

print("\n1️⃣ Database file still exists after closing:")
print(f"   ✅ File exists: {os.path.exists(DB_FILE)}")
print(f"   📁 Location: {os.path.abspath(DB_FILE)}")
print(f"   💾 Size: {os.path.getsize(DB_FILE):,} bytes")

print("\n2️⃣ Can read data without agent:")
verify_conn = sqlite3.connect(DB_FILE)
cursor = verify_conn.cursor()
cursor.execute("SELECT COUNT(*) FROM checkpoints WHERE thread_id = ?", 
               (config['configurable']['thread_id'],))
thread_checkpoints = cursor.fetchone()[0]
print(f"   ✅ Found {thread_checkpoints} checkpoints for thread: {config['configurable']['thread_id'][:30]}...")
cursor.close()
verify_conn.close()

print("\n3️⃣ Data survives Python kernel restart:")
print(f"   ✅ You can restart this notebook and reconnect to: {DB_FILE}")
print(f"   ✅ All conversation history will be preserved")
print(f"   ✅ Can resume from ANY checkpoint in the history")

print("\n" + "="*80)
print("✅ VERIFICATION COMPLETE - Persistence is working!")
print("="*80)

---
# 📚 Summary and Key Takeaways

## 🎯 What We Demonstrated

### Session 1 (Day 1):
✅ Created file-based SQLite database  
✅ Started employee onboarding conversation  
✅ Verified documents  
✅ Assigned equipment  
✅ Saved state to disk  
✅ Closed application  

### Session 2 (Day 2):
✅ Reconnected to same database file  
✅ Retrieved complete conversation history  
✅ Continued onboarding seamlessly  
✅ Agent remembered all previous context  
✅ Completed remaining steps  

## 🔑 Key Concepts

### 1. File-Based vs In-Memory Persistence

| Feature | `:memory:` | File-Based |
|---------|-----------|------------|
| Storage | RAM only | Disk file |
| Survives restart | ❌ No | ✅ Yes |
| Use case | Testing | Production |
| Speed | Faster | Slightly slower |

### 2. Thread IDs
- Unique identifier for each conversation
- Use same thread_id to resume conversation
- Different thread_ids = isolated conversations

### 3. Checkpoints
- State snapshots saved automatically
- One checkpoint per agent turn
- Can retrieve and resume from any checkpoint

### 4. System Messages in LangChain 1.0
- No `prompt=` parameter in `create_agent()`
- Pass system message in first invoke
- Checkpointer remembers system message

## 🚀 Production Benefits

**Real-World Applications:**
- Multi-day onboarding processes
- Long-running support conversations
- Human-in-the-loop approvals
- Audit trails and compliance
- Resume after system failures

## 💡 Best Practices

1. **Always use file-based SQLite for production**
2. **Use descriptive thread IDs** (e.g., `employee_{id}_onboarding_{year}`)
3. **Close database connections properly**
4. **Implement backup strategy** for database files
5. **Consider PostgreSQL** for multi-user scenarios

## 📖 Next Steps

**To use in production:**
- Add error handling for database operations
- Implement connection pooling
- Add authentication/authorization for threads
- Set up database backups
- Monitor database size and performance
- Consider PostgreSQL for scalability

**Advanced topics:**
- State forking and branching
- Time-travel debugging
- Human-in-the-loop workflows
- Multi-agent state sharing

---

## 🎓 Exercises

### Exercise 1: Multiple Employees
Modify this notebook to handle 3 different employees simultaneously using different thread IDs.

### Exercise 2: State Inspection
Create a function that lists all active onboarding threads in the database.

### Exercise 3: Resume from Specific Checkpoint
Use `checkpoint_id` to resume from a specific point in the conversation, not just the latest.

### Exercise 4: Error Recovery
Add try-except blocks to handle database connection errors and implement retry logic.

### Exercise 5: Migration to PostgreSQL
Research and document how to migrate this code to use PostgreSQL instead of SQLite.

---

## 📞 Support

Questions or issues? Check:
- LangChain Documentation: https://python.langchain.com/
- LangGraph Documentation: https://langchain-ai.github.io/langgraph/
- SQLite Documentation: https://www.sqlite.org/docs.html

---

**🎉 Congratulations!** You've successfully built a production-ready agent with file-based persistence and pause/resume capabilities!